In [ ]:
%run config.py

In [ ]:
from scipy.constants import elementary_charge as qe, electron_mass as me, proton_mass as mp, mu_0 as mu0
from ford1991 import solve_3d
from mpl_toolkits import mplot3d
from matplotlib import ticker
import matplotlib.colors as colors

## Create an approximate magnetic bottle by combining two dipole fields.

Concept from:

https://uio-ccse.github.io/computational-essay-showroom/essays/exercises/essayseeds/MagneticBottle_Numpy.html

Could be made faster by using JIT(compilation) from numba, as in original source.
Computing arrays of field values, the numpy implementation below is quicker, however for large numbers of pointwise
calculations (as needed by particle push), the jit option is faster.

In [ ]:
# Magnetic moment vector
mu = 1e4 * np.array([0,0,1])

# Switch off Larmor term
tau = 0.0

# 3D domain
x = np.linspace(-10,10,201)
y = np.linspace(-10,10,201)
z = np.linspace(-10,10,201)

In [ ]:
def dipole_field(x, y, z, mu, z0=0.0):
    # Coordinates with origin defined at z0
    _x = x
    _y = y
    _z = z - z0
    
    X, Y, Z = np.meshgrid(_x, _y, _z, indexing='ij')

    # Array of positions
    r = np.array([X, Y, Z])
    
    rmag2 = np.sum(r*r, axis=0)
    rmag = np.sqrt(rmag2)
    rmag3 = rmag2 * rmag + 1e-30
    rmag5 = rmag3 * rmag2 + 1e-30
    
    # Perhaps there are nicer ways to do this?
    mudotr = mu[0] * r[0,:,:,:] + mu[1] * r[1,:,:,:] + mu[2] * r[2,:,:,:]

    b1 = 3.0 * r * mudotr / rmag5
    b2 = - np.array([mu[0] / rmag3, mu[1] / rmag3, mu[2] / rmag3])
    
    return  np.squeeze(b1 + b2) * (mu0 / 4.0 / np.pi)

In [ ]:
def bottle_field(x, y, z, mu=1e4 * np.array([0,0,1])):
    b1 = dipole_field(x, y, z, mu, z0=10)
    b2 = dipole_field(x, y, z, mu, z0=-10)
    
    return b1 + b2

In [ ]:
btot = bottle_field(x, y, z, mu)

# Plot field at x = 0
x0 = np.argmin(np.abs(x))
Y, Z = np.meshgrid(y, z)
# Transpose for plotting
by = np.transpose(btot[1,x0,:,:])
bz = np.transpose(btot[2,x0,:,:])
plt.streamplot(Y, Z, by, bz)
plt.xlim(-10.0,10.0)
plt.ylim(-10.0,10.0)

In [ ]:
# Test for trace alpha particle
m = 4.0*mp
q = 2.0*qe

# Initial conditions
r0 = np.array([0.0, -5.0, 0.0])
v0 = np.array([0.0, 0.0, 100.0])

In [ ]:
%%time

b0 = bottle_field(r0[0], r0[1], r0[2], mu)
ic = (r0[0], r0[1], r0[2], v0[0], v0[1], v0[2], 0.0)
res = solve_3d(25, b0=b0, v0=v0, mass=m, charge=q, tau=tau, calc_b_field=bottle_field, ic=ic)
print('Final simulated time = %.4fs' % res.t[-1])

In [ ]:
# Plot field through x=0 plane
by = np.transpose(btot[1,x0,:,:])
bz = np.transpose(btot[2,x0,:,:])
plt.streamplot(Y,Z, by, bz, color="black")
plt.plot(res.y[1], res.y[2])
plt.xlim(-10.0,10.0)
plt.ylim(-10.0,10.0)
plt.xlabel("$y$")
plt.ylabel("$z$")
plt.gcf().set_size_inches([8,8])

In [ ]:
%%time

# Test for electron.
m = me
q = -qe

# Increase initial velocity
v0 = np.array([0.0, 0.0, 5e5])

ic = (r0[0], r0[1], r0[2], v0[0], v0[1], v0[2], 0.0)
res = solve_3d(25, b0=b0, v0=v0, mass=m, charge=q, tau=tau, calc_b_field=bottle_field, ic=ic)
print('Final simulated time = %.4fs' % res.t[-1])

In [ ]:
# Plot x=0 plane
plt.streamplot(Y,Z, by, bz, color="black")
plt.plot(res.y[1], res.y[2])
plt.xlim(-10.0,10.0)
plt.ylim(-10.0,10.0)
plt.xlabel("$y$")
plt.ylabel("$z$")
plt.gcf().set_size_inches([8,8])

In [ ]:
# Now plot y = 0 plane
X, Z = np.meshgrid(x, z)
y0 = np.argmin(np.abs(y))
bx = np.transpose(btot[0,:,y0,:])
bz = np.transpose(btot[2,:,y0,:])
plt.streamplot(X,Z, bx, bz, color="black")
plt.plot(res.y[0], res.y[2])
plt.xlim(-10.0,10.0)
plt.ylim(-10.0,10.0)
plt.xlabel("$x$")
plt.ylabel("$z$")
plt.gcf().set_size_inches([8,8])

In [ ]:
# Trace magnetic field lines starting at (x0, y0, z0)
def trace_field(x0, y0, z0, zmax=7.5, nmax=1000, eps=1e-1):
    x = [x0]
    y = [y0]
    z = [z0]
    
    for i in range(nmax):
        b = bottle_field(x[-1], y[-1], z[-1])
        b /= np.linalg.norm(b)
        x.append(x[-1] + b[0]*eps)
        y.append(y[-1] + b[1]*eps)
        z.append(z[-1] + b[2]*eps)
        if np.abs(z[-1]) > zmax:
            break
        
    return [x, y, z]

In [ ]:
# A not very general purpose field tracing routine.
# Ideally zmax = z0, but few field lines make it into the domain
# More generally would be better to automatically move into domain
# rather than using eps to switch signs.
def get_field_lines(eps=1e-2, nmax=100000, zmax = 9.0, z0=7.5):
    lines = []
    # Sample these points + origin, from top and bottom of domain
    rad = np.linspace(0.5, 3, 6)
    theta = np.linspace(0, 2*np.pi*9/10, 10)

    lines.append(trace_field(0,0,-z0, zmax=zmax, eps=eps, nmax=nmax))
    for r in rad:
        for t in theta:
            x0 = r * np.cos(t)
            y0 = r * np.sin(t)
            lines.append(trace_field(x0,y0,-z0, zmax=zmax, eps=eps, nmax=nmax))

    # Now trace lines from top
    lines.append(trace_field(0,0,z0, zmax=zmax, eps=-eps, nmax=nmax))
    for r in rad:
        for t in theta:
            x0 = r * np.cos(t)
            y0 = r * np.sin(t)
            lines.append(trace_field(x0,y0,z0, zmax=zmax, eps=-eps, nmax=nmax))
            
    return lines

In [ ]:
zlim = 9.0
lines = get_field_lines(zmax = zlim)

In [ ]:
fig = plt.figure(figsize =(20, 20))
ax = plt.axes(projection ='3d')
cmap = plt.get_cmap('Oranges')
X2, Y2 = np.meshgrid(x,y)

# Bottom layer
z0 = -zlim
iz0 = np.argmin(np.abs(z - z0))
vmin = np.min(-btot[2,:,:,iz0])
vmax = np.max(-btot[2,:,:,iz0])
vmax = max(vmax, -vmin)
cset = plt.contourf(X2, Y2, btot[2,:,:,iz0], cmap=cmap, alpha=0.5, vmin=-vmax, vmax=vmax, zdir='z', offset=z0)

# Top layer
z0 = zlim
iz0 = np.argmin(np.abs(z - z0))
vmin = np.min(btot[2,:,:,iz0])
vmax = np.max(btot[2,:,:,iz0])
# Symmetric limit look nicer for plotting
vmax = max(vmax, -vmin)
cset = plt.contourf(X2, Y2, btot[2,:,:,iz0], cmap=cmap, alpha=0.5, vmin=-vmax, vmax=vmax, zdir='z', offset=z0)

fig.colorbar(cset, fraction=0.025, pad=0)

for l in lines:
    ax.plot(l[0],l[1],l[2], color='blue', alpha=0.25)
    
ax.plot(res.y[0], res.y[1], res.y[2], color='red')

ax.set_xlim3d(-10, 10)
ax.set_ylim3d(-10, 10)
ax.set_zlim3d(-zlim, zlim)
ax.view_init(elev=15, azim=45)